In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0857083797454834
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 31.375944500220307
200000 53.61824416955852
300000 58.07392553528623
400000 57.658233572185665
500000 54.08180512978326
600000 54.44096923725643
700000 61.71663821844767
800000 67.71716123158846
900000 74.36060670226242
1000000 86.25094694359476
1100000 83.14238286350489
1200000 90.28188457870053
1300000 31.75724363163441
1400000 91.41565893980757
1500000 98.49309719358463
1600000 97.3963838891516
1700000 97.35079223709307
1800000 103.90818284926814
1900000 111.98299340150493
2000000 111.63470045884601
2100000 105.85846576692961
2200000 103.04810252522194
2300000 109.44165095040982
2400000 105.79459138662666
2500000 94.5796389147004
2600000 33.71584876164856
2700000 102.4452167922941
2800000 104.68907733504606
2900000 100.77371223469297
3000000 97.2289775

29400000 108.84873191885272
29500000 102.96071132045483
29600000 106.21622738940208
29700000 107.43186042859013
29800000 104.38785199930152
29900000 105.42933808259517
30000000 71.87852450329395
30100000 82.98671948315165
30200000 109.8962635741955
30300000 113.18328684191128
30400000 100.56307653404293
30500000 60.48193718806488
30600000 117.7419609767596
30700000 116.92498032273846
30800000 121.7057266989553
30900000 110.2184078068607
31000000 111.56848891106242
31100000 107.9337564304347
31200000 110.10076412495988
31300000 107.29930130685177
31400000 102.52219267072823
31500000 20.534899797858102
31600000 108.5598470285594
31700000 106.50217588942854
31800000 105.18440669223611
31900000 111.88339149736717
32000000 111.22214615309002
32100000 108.17491216870268
32200000 117.46254777986837
32300000 115.15414314275097
32400000 110.11995821030486
32500000 64.80756278333001
32600000 89.53498550073867
32700000 117.3786423764804
32800000 110.1174206024925
32900000 112.7297249566762
330000

59200000 45.517339585166695
59300000 95.5490488305968
59400000 93.16795990483627
59500000 102.0037223724017
59600000 99.3338937629721
59700000 103.47908429771097
59800000 102.64733108485035
59900000 104.72073679089013
60000000 35.81447800352638
60100000 114.63488998245336
60200000 107.6517524407445
60300000 99.90943699419668
60400000 108.02105472621973
60500000 109.16116970449941
60600000 108.54849356698261
60700000 106.99847361697658
60800000 96.5157151823115
60900000 54.59415637375452
61000000 106.94928811428187
61100000 103.84699121807428
61200000 103.44988765297832
61300000 102.85758709556512
61400000 97.04505898690276
61500000 102.56102685330698
61600000 87.76939185263498
61700000 28.41890357713475
61800000 100.21227562378846
61900000 98.26343401375577
62000000 96.39399077843443
62100000 104.13144383862088
62200000 97.64703688483857
62300000 102.16730728046349
62400000 103.3617367320145
62500000 35.604286039626224
62600000 112.95451804101734
62700000 106.11394732156644
62800000 10

89300000 61.500833144874434
89400000 53.667449071506134
89500000 84.76264103707352
89600000 90.74488184940977
89700000 84.07485212906236
89800000 91.34133260988618
89900000 72.64743937000729
90000000 58.01507892994403
90100000 93.5657208378675
90200000 94.39399353595984
90300000 94.67610672946302
90400000 97.64564791349629
90500000 104.80134985683372
90600000 95.46235146953562
90700000 102.89472213470879
90800000 103.54148504318313
90900000 101.66376605329478
91000000 99.3460223378137
91100000 101.41570580130082
91200000 89.65501782235648
91300000 42.01542029450687
91400000 96.30108867711296
91500000 91.40639464222534
91600000 97.42147816412775
91700000 94.22324047741687
91800000 75.64299877435279
91900000 40.30783233152583
92000000 93.07632187754973
92100000 91.85416566983642
92200000 89.23409029774204
92300000 92.20114293277969
92400000 74.32578046241981
92500000 59.89432902760395
92600000 97.3463071303309
92700000 95.09961135922926
92800000 104.19954767876467
92900000 104.6775933642

118900000 91.78129510840706
119000000 91.12286304190397
119100000 66.62714955807196
119200000 56.93692050485651
119300000 90.53943213139016
119400000 84.22628234022073
119500000 52.21283738394411
119600000 62.21670289662716
119700000 86.11208515769036
119800000 88.39873219555007
119900000 50.3351603458806
120000000 73.07684394015683
120100000 88.24665725670101
120200000 90.01017062098185
120300000 85.52105947085187
120400000 94.85635118750479
120500000 96.98819863833533
120600000 98.52599951427516
120700000 99.98869372173289
120800000 103.02130391164674
120900000 107.8995636751874
121000000 106.99388674687093
121100000 106.92555981125662
121200000 103.77206617161117
121300000 91.71703151128044
121400000 96.73678137053588
121500000 93.76666493657133
121600000 93.82172398061135
121700000 31.381940658235163
121800000 96.23659603790385
121900000 98.6960452903811
122000000 65.83474128746772
122100000 53.15505823446335
122200000 92.50464051897526
122300000 94.82145441855313
122400000 54.9823

...

952800000 0.5867481589350382
952900000 1.058910507671378
953000000 1.6763186782350161
953100000 2.3813971314424713
953200000 1.3582023787221327
953300000 1.7156968620158022
953400000 0.7495939403627813
953500000 0.7718552596517742
953600000 1.425398259983384
953700000 1.5910780972840906
953800000 1.350092075391284
953900000 1.2838021731618696
954000000 0.9402037125081281
954100000 0.5888423240183387
954200000 1.3599663087105782
954300000 1.5374432276580692
954400000 1.4725973492550815
954500000 1.5001886743215018
954600000 1.2263465648930807
954700000 0.5479122528101484
954800000 1.2010541413093117
954900000 1.4311522734245559
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.5229078449069493
955100000 1.3507110548631098
955200000 1.7494928339627802
955300000 1.0781250158932203
955400000 1.1614371441806137
955500000 1.8770844865309377
955600000 1.4773091507797547
955700000 1.3500209793978466
955800000 1.2505537599026322
955900000 1.2849696805850712
95600

979100000 0.4751166610107005
best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.31857383373177844
979300000 0.351039444844993
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.3197259996242861
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3322260734284103
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.6757155944043662
979700000 2.0161418363237553
979800000 1.9493911531862518
979900000 2.111754776735784
980000000 1.4006955883862389
980100000 1.6499066793670993
980200000 1.938441248357122
980300000 1.3295472681013802
980400000 1.144661524082905
980500000 1.2886930356519342
980600000 2.110769071582624
980700000 1.6133941610657296
980800000 2.686254116569063
98090

998400000 0.8712487589436292
998500000 1.2214012566195218
998600000 1.434911155929888
998700000 0.9002615166384875
998800000 1.1802316829831738
998900000 1.3548717812393771
999000000 1.0810864594250222
999100000 1.1720813092634288
999200000 1.4137712978149235
999300000 1.1183237317933439
999400000 1.0688412321056804
999500000 2.0582804963104646
999600000 1.1765693543902678
999700000 0.9511825600047595
999800000 1.2979490876330217
999900000 1.4048587986487509
1000000000 0.832134180373699
1000100000 1.2717759346236719
1000200000 1.3699456642661512
1000300000 1.0119969963549145
1000400000 1.4689058716370385
1000500000 0.24397817119561435
1000600000 0.24217991270699898
1000700000 0.24250662067616183
1000800000 0.28723372902684413
1000900000 0.3591036465640101
1001000000 0.28650679024617076
1001100000 0.24253422562545537
1001200000 0.23931794088571273
1001300000 0.23851201027624672
1001400000 0.23801332306391004
1001500000 0.2381900769365264
1001600000 0.23619173306196
1001700000 0.23640068

1025300000 0.36695989498279824
1025400000 0.27362222151571514
1025500000 0.4645922289999472
1025600000 0.5974325452794632
1025700000 0.24381487116059267
1025800000 0.2231508863541517
1025900000 0.30209317428306875
1026000000 0.26453871486081676
1026100000 0.19458974988866845
1026200000 0.4508541528709114
1026300000 0.292888518272376
1026400000 0.16314681957268384
1026500000 0.3104013720143868
1026600000 0.2878524934565345
1026700000 0.17778215532530547
1026800000 0.33959010017328395
1026900000 0.2852291041034897
1027000000 0.2023686075351225
1027100000 0.27964979910101756
1027200000 0.2834215840126455
1027300000 0.22269882460355425
1027400000 0.2557687233634366
1027500000 0.44298129122502605
1027600000 0.21650958335713547
1027700000 0.2564833347830587
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.30565712400515077
1027900000 0.264026271073244
1028000000 0.24294662960847618
1028100000 0.2580661673066199
1028200000 0.286339886760917
1028300000 0.25615

best so far: 0
type: [2, 34, 1, 2, 1] 136
cases of this type: 272
best so far: 0
type: [2, 34, 2, 1, 1] 136
cases of this type: 136
best so far: 0
type: [2, 68, 1, 1, 1] 136
cases of this type: 68
best so far: 0
type: [4, 1, 1, 1, 34] 136
cases of this type: 1336336
1045400000 0.04037606320050425
1045500000 0.050823573708677294
1045600000 0.04745464771216247
1045700000 0.0405550474266039
1045800000 0.07576192096365889
1045900000 0.03506272286431458
1046000000 0.027651463606517844
1046100000 0.01174432815733552
1046200000 0.039174715331684214
1046300000 0.02996333507907192
1046400000 0.037337809821605686
1046500000 0.028564208173571033
1046600000 0.029770948371986548
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.028684415445607903
1046800000 0.01457052572231094
1046900000 0.01363623186650806
1047000000 0.010548961044752598
1047100000 0.005218700669895278
1047200000 0.008493153210398223
1047300000 0.009119896501002709
best so far: 0
type: [4, 1, 1, 17,